# Caution !!! 
I heve not checked bugs yet.

# Two-leg ladder spin-$1/2$ Heisenberg model

This program uses the single-site unit cell VUMPS algorithm [1].

This program corresponds to Fig. 5 in Ref. [3].

#### Reference

[1] Phys. Rev. B 97, 045145 (2018)

https://journals.aps.org/prb/abstract/10.1103/PhysRevB.97.045145

https://arxiv.org/abs/1701.07035

[2] Phys. Rev. B 86, 125441 (2012)

https://journals.aps.org/prb/abstract/10.1103/PhysRevB.86.125441

https://arxiv.org/abs/1204.0704

[3] Phys. Rev. B 85, 075125 (2012)

https://journals.aps.org/prb/abstract/10.1103/PhysRevB.85.075125

https://arxiv.org/abs/0909.4059

In [2]:
import numpy as np
import scipy as sp
from scipy.sparse.linalg import LinearOperator

import sys
sys.path.append('Library')
import MathFunctions as MF
import MPSOperators as MO
import SingleVUMPS as SV

In [11]:
dtype = np.dtype("float") 
#dtype = np.dtype("complex")

D = 2 # physical bond D = 2 corresponds to spin-1/2
M = 30 # virtual bond  
max_iter = 500 # the number of the VUMPS steps
D2 = D * D 

Sx,Sy,Sz,Su,Sd = MF.Spin(D)
Sz2 = np.einsum("ab,bc -> ac",Sz,Sz)

Jleg = 1.0; Jrung = 1.0;
h_xxx = ( np.kron(Sx,Sx) + np.kron(Sy,Sy) + np.kron(Sz,Sz) ).real
I = np.eye(D*D)
h = 0.5 * Jrung * np.kron(h_xxx,I).reshape(D2,D2,D2,D2) \
    + 0.5 * Jrung * np.kron(I,h_xxx).reshape(D2,D2,D2,D2) \
    + Jleg * np.kron(h_xxx,I).reshape(D,D,D,D,D,D,D,D).transpose(0,2,1,3,4,6,5,7).reshape(D**4,D**4).reshape(D2,D2,D2,D2) \
    + Jleg * np.kron(I,h_xxx).reshape(D,D,D,D,D,D,D,D).transpose(0,2,1,3,4,6,5,7).reshape(D**4,D**4).reshape(D2,D2,D2,D2)

In [13]:
# initial state
np.random.seed(3162277)
A = np.random.rand(M,D2,M) 
HR = np.random.rand(M,M); HL = np.random.rand(M,M)
tol = 1e-10

AC,C,AR,AL = MO.MixedCanonicalForm(A,dtype)

for i in range (max_iter):
    HR,er = SV.Calc_HR(AR,HR,h,dtype,tol=tol)
    HL,el = SV.Calc_HL(AL,HL,h,dtype,tol=tol)
    AC = SV.Next_AC(AC,AR,AL,HR,HL,h,dtype,tol=tol)
    C = SV.Next_C(C,AR,AL,HR,HL,h,dtype,tol=tol)
    #AR = SV.Next_AR_SVD(AC,C)
    AR = SV.Next_AR_PolarDecomposition(AC,C)
    #AL = SV.Next_AL_SVD(AC,C)
    AL = SV.Next_AL_PolarDecomposition(AC,C)
    B = SV.Calc_B(AC,C,AR,AL)
    tol = B / 100
    if B < 1e-12: 
        print ("Converged!")
        break
    E = 0.5 * ( er + el ).real
    print ("step {}, E {}, |B| {}".format(i,E/2,B))

#Tensors = np.empty(4, dtype=object)
#Tensors[0] = AC; Tensors[1] = C; Tensors[2] = AR; Tensors[3] = AL
#np.save("Data/Tensors_Heisenberg_{}_{}_{}_{}_{}.npy".format(D,M,JJ,BB,DD),Tensors)

step 0, E 0.3747957383451404, |B| 0.6901806392842745
step 1, E -0.4612725447683895, |B| 0.06960191599807383
step 2, E -0.577781576589631, |B| 0.0009100835238643438
step 3, E -0.5780400087111095, |B| 0.000246792654496597
step 4, E -0.5780420727775445, |B| 0.0001562412785329729
step 5, E -0.5780421957134088, |B| 0.00016645682931763095
step 6, E -0.5780421952743988, |B| 0.00018238592750027382
step 7, E -0.5780421870690309, |B| 0.00018652472074347367
step 8, E -0.5780421977346266, |B| 0.00018773207064901848
step 9, E -0.5780422158046312, |B| 0.00017708586219277044
step 10, E -0.5780422368773837, |B| 0.00016986518790324093
step 11, E -0.5780422548967421, |B| 0.00016569172879399196
step 12, E -0.5780422695940132, |B| 0.0001646029161318995
step 13, E -0.5780422811436825, |B| 0.0001668291579029103
step 14, E -0.578042289845023, |B| 0.00017163751353287062
step 15, E -0.5780422962500299, |B| 0.00016765130351065126
step 16, E -0.5780423329285013, |B| 0.0001534460008433905
step 17, E -0.5780423571

step 146, E -0.5780425406509639, |B| 2.70216149173714e-08
step 147, E -0.5780425406509713, |B| 2.561593847220252e-08
step 148, E -0.5780425406509773, |B| 2.4283387131429127e-08
step 149, E -0.5780425406509826, |B| 2.3020156661211584e-08
step 150, E -0.5780425406509851, |B| 2.1822640851215335e-08
step 151, E -0.5780425406509909, |B| 2.0687421069840537e-08
step 152, E -0.5780425406509953, |B| 1.9611256524613017e-08
step 153, E -0.5780425406509985, |B| 1.8591075025407393e-08
step 154, E -0.5780425406510004, |B| 1.7623964223638315e-08
step 155, E -0.5780425406510037, |B| 1.6707163309002285e-08
step 156, E -0.5780425406510062, |B| 1.5838055060398442e-08
step 157, E -0.5780425406510067, |B| 1.5014158472854004e-08
step 158, E -0.5780425406510104, |B| 1.4233121546972619e-08
step 159, E -0.5780425406510119, |B| 1.3492714660950286e-08
step 160, E -0.5780425406510141, |B| 1.279082419755501e-08
step 161, E -0.5780425406510132, |B| 1.212544640675842e-08
step 162, E -0.5780425406510177, |B| 1.149468

step 285, E -0.5780425406510283, |B| 1.614111570908943e-11
step 286, E -0.5780425406510281, |B| 1.5303015806320226e-11
step 287, E -0.5780425406510283, |B| 1.4508574018893777e-11
step 288, E -0.5780425406510288, |B| 1.3755265592578183e-11
step 289, E -0.5780425406510281, |B| 1.3041324490425098e-11
step 290, E -0.578042540651029, |B| 1.2364422041717788e-11
step 291, E -0.5780425406510284, |B| 1.1722755975002289e-11
step 292, E -0.5780425406510298, |B| 1.1114428565658383e-11
step 293, E -0.5780425406510281, |B| 1.0537706784336344e-11
step 294, E -0.5780425406510292, |B| 9.991005624822194e-12
step 295, E -0.578042540651029, |B| 9.472699885895797e-12
step 296, E -0.5780425406510297, |B| 8.981393710279494e-12
step 297, E -0.5780425406510299, |B| 8.51559177052841e-12
step 298, E -0.578042540651029, |B| 8.073982784019455e-12
step 299, E -0.5780425406510294, |B| 7.655330962648957e-12
step 300, E -0.5780425406510286, |B| 7.2584356922431036e-12
step 301, E -0.5780425406510283, |B| 6.881987205224